In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score,recall_score

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback
from tensorflow.nn import sigmoid_cross_entropy_with_logits
from tensorflow.metrics import precision
import tensorflow.keras.backend as K
import tensorflow as tf

In [28]:
from tensorflow.keras.metrics import binary_accuracy,mae

In [3]:
# X=[]
# img_paths = os.listdir("cqts2")
# for i in range(len(img_paths)):
#     img = cv2.imread("cqts2/"+img_paths[i],0)
#     suff = img[4:235,35:-4]
#     if(suff.shape==(231,371)):
#         for j in range(suff.shape[1]-6):
#             suff2 = suff[:,j:j+7]
#             suff2 = suff2.reshape(7,231,1)
# #             if(i==0 and j==0):
# #                 X = suff2
# #             else:
# #                 X = np.concatenate((X,suff2),axis=0)
#             X.append(suff2)
#     if(i%10==0):
#         print(i)

In [4]:
X=[]
img_paths = os.listdir("cqts2")
for i in range(len(img_paths)):
    img = cv2.imread("cqts2/"+img_paths[i],0)
    suff = img[4:235,35:-4]
    if(suff.shape==(231,371)):
        for j in range(suff.shape[1]//7):
            suff2 = suff[:,j*7:(j+1)*7]
            suff2 = suff2.reshape(7,231,1)
#             if(i==0 and j==0):
#                 X = suff2
#             else:
#                 X = np.concatenate((X,suff2),axis=0)
            X.append(suff2)
    if(i%10==0):
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


In [15]:
X = np.array(X)

In [16]:
X.shape

(11395, 7, 231, 1)

In [7]:
# label_paths = os.listdir("labels2")
# count=0
# for i in range(len(label_paths)):
#     img = cv2.imread("cqts2/"+img_paths[i],0)
#     suff = img[4:235,35:-4]
#     if(suff.shape==(231,371)):
#         count+=1
#         lbl = np.load("labels2/"+label_paths[i])
#         lbl2 = lbl[:,:-4]
#         for j in range(lbl2.shape[1]-6):
#             chunk_sum = np.sum(lbl2[:,j:j+7],axis=1)
#             for k in range(len(chunk_sum)):
#                 if(chunk_sum[k]>3):
#                     chunk_sum[k]=1
#                 else:
#                     chunk_sum[k]=0
#             if(i==0 and j==0):
#                 y = chunk_sum.reshape(1,88)
#             else:
#                 y = np.concatenate((y,chunk_sum.reshape(1,88)),axis=0)
#     if(i%10==0):
#         print(i)

In [68]:
label_paths = os.listdir("labels2")
count=0
cc=0
X=[]
img_paths = os.listdir("cqts2")
for i in range(len(img_paths)):
#     img = cv2.imread("cqts2/"+img_paths[i],0)
#     suff = img[4:235,35:-4]
    try:
        img = cv2.imread("cqts2/"+img_paths[i],0)
        suff = img[4:235,35:-4]
        if(suff.shape==(231,371)):
            count+=1
            try:
                lbl = np.load("labels2/"+img_paths[i][:-8]+'.npy')
            except:
                lbl=np.load("labels2/"+img_paths[i][:-4]+'.npy')
            lbl2 = lbl[:,:-4]
            for j in range(lbl2.shape[1]//7):
                chunk_sum = np.sum(lbl2[:,j*7:(j+1)*7],axis=1)
                for k in range(len(chunk_sum)):
                    if(chunk_sum[k]>4):
                        chunk_sum[k]=1
                    else:
                        chunk_sum[k]=0
                if(i==0 and j==0):
                    y = chunk_sum.reshape(1,88)
                else:
                    y = np.concatenate((y,chunk_sum.reshape(1,88)),axis=0)
            for j in range(suff.shape[1]//7):
                suff2 = suff[:,j*7:(j+1)*7]
                suff2 = suff2.reshape(7,231,1)
                X.append(suff2)
        if(cc%10==0):
            print(i)
        cc+=1
    except:
        pass

0
10
20
30
42
52
62
72
82
92
103
113
123
133
143
153
163
173
183
193
203
213


In [69]:
X=np.array(X)
X.shape

(11236, 7, 231, 1)

In [73]:
# label_paths[i]
i

218

In [74]:
count

215

In [70]:
y.shape
y2=y[:,27:70]

In [71]:
y2.shape

(11236, 43)

In [72]:
# np.save("y_successive.npy",y)

In [73]:
len(img_paths)

218

In [74]:
lbl.shape

(88, 375)

In [75]:
# X = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
# X.shape

In [76]:
y2 = y2.astype(int)
y=y.astype(int)

In [77]:
y_ = y*100

In [78]:
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [79]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.matthews_correlation = []

    def on_epoch_end(self, batch, logs={}):
        self.matthews_correlation.append(logs.get('matthews_correlation'))

def matthews_correlation(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def precision(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    #tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    #fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp)
    denominator = (tp + fp)

    return numerator / denominator

def recall(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    #tn = K.sum(y_neg * y_pred_neg)

    #fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp)
    denominator = (tp + fn)

    return numerator / denominator

def f1score(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    #tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp)
    denominator1 = (tp + fp)
    denominator2 = (tp + fn)

    preci =  numerator / denominator1
    rec = numerator / denominator2
    
    f1 = (2*preci*rec)/(preci+rec)
    return f1

def network():
    model = Sequential()
    model.add(Conv2D(filters=50,kernel_size=(5,15),input_shape=(7,231,1),activation="sigmoid"))
    model.add(MaxPool2D(pool_size=(1,3)))
    model.add(Dropout(0.4))
    model.add(Conv2D(filters=50,kernel_size=(3,5),activation="sigmoid"))
    model.add(MaxPool2D(pool_size=(1,3)))
    model.add(Dropout(0.4))
    model.add(Flatten())
#     model.add(Dense(2000,activation="sigmoid"))
#     model.add(Dropout(0.3))
    model.add(Dense(1000,activation="sigmoid"))
    model.add(Dropout(0.4))
    model.add(Dense(200,activation="relu"))
#     model.add(Dropout(0.3))
    model.add(Dense(43,activation="sigmoid"))
    
    opt = SGD(lr=0.01,momentum=0.9)
#     opt = SGD()
#     lossf = sigmoid_cross_entropy_with_logits()
#     met = precision()
    model.compile(optimizer=opt,loss="binary_crossentropy", metrics=[matthews_correlation,binary_accuracy,mae])
#     model.compile(optimizer="adam",loss="mean_squared_error", metrics=[matthews_correlation])
    return model

In [80]:
model = network()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 3, 217, 50)        3800      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 3, 72, 50)         0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 3, 72, 50)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 1, 68, 50)         37550     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 1, 22, 50)         0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 1, 22, 50)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1100)              0         
__________

In [81]:
X_train,X_test,y_train,y_test = train_test_split(X,y2,test_size=0.1)

In [82]:
history = LossHistory()
for i in range(5):
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1,batch_size=100,shuffle=True,callbacks=[history])
    v_pred = model.predict(X_test)
    for j in range(v_pred.shape[0]):
        ind= v_pred[j].argsort()[-5:][::-1]
#         print(v_pred[])
        v_pred[j]=np.zeros((v_pred[j].shape[0]))
        v_pred[j][ind]=1
#     print(v_pred[0])
#     break
#     arr.argsort()[-3:][::-1]
# Out[3]: array([4, 3, 1])
#     v_pred1 = np.where(v_pred>.2,1,0)
#     print(y_test.flatten().shape,v_pred1.flatten().shape)
    print('precision',precision_score(y_test.flatten(),v_pred.flatten()))
    print('recall',recall_score(y_test.flatten(),v_pred.flatten()))
    print (i,f1_score(y_test.flatten(),v_pred.flatten()))    

Train on 10112 samples, validate on 1124 samples
Epoch 1/1
10112/10112 [==============================] - 6s 604us/step - loss: 0.2444 - matthews_correlation: 0.0065 - binary_accuracy: 0.9145 - mean_absolute_error: 0.1420 - val_loss: 0.1713 - val_matthews_correlation: 0.0000e+00 - val_binary_accuracy: 0.9520 - val_mean_absolute_error: 0.0875
precision 0.14270462633451958
recall 0.3455407152089617
0 0.20198967384460395
Train on 10112 samples, validate on 1124 samples
Epoch 1/1
10112/10112 [==============================] - 5s 489us/step - loss: 0.1770 - matthews_correlation: 5.3691e-04 - binary_accuracy: 0.9514 - mean_absolute_error: 0.0889 - val_loss: 0.1708 - val_matthews_correlation: 0.0000e+00 - val_binary_accuracy: 0.9520 - val_mean_absolute_error: 0.0882
precision 0.14288256227758006
recall 0.3459715639810427
1 0.202241531293288
Train on 10112 samples, validate on 1124 samples
Epoch 1/1
10112/10112 [==============================] - 5s 529us/step - loss: 0.1765 - matthews_correlat

In [51]:
for j in range(v_pred.shape[0]):
    ind= v_pred[j].argsort()[-5:][::-1]
#     print(j)
#     print(v_pred[j].shape)
    v_pred[j]=np.zeros((v_pred[j].shape[0]))
    v_pred[j][ind]=1

In [63]:
v_pred[45]
# ind

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [36]:
np.where(v_pred[7]>0.26,1,0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [53]:
y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [119]:
np.max(v_pred,axis=1)

array([0.30129522, 0.29945564, 0.29238912, ..., 0.31104252, 0.3162166 ,
       0.30906853], dtype=float32)

In [87]:
y_pred = model.predict(X_test)

In [102]:
np.where(y_pred[45]>=11.0/88)

(array([36, 48, 50, 53, 55], dtype=int64),)

In [103]:
np.where(y_test[45]==1)[0]

array([44, 46, 50, 55], dtype=int64)